In [1]:
import torch
from typing import Tuple


# LAB SHEET 1

############
# PART 1.1 #
############

def sgd_factorise(A: torch.Tensor , rank:int , num_epochs=1000, lr =0.01):# −> Tuple[torch.Tensor, torch.Tensor]:
    m = A.shape[0]
    n = A.shape[1]
    U = torch.rand(m,rank)
    V = torch.rand(n,rank)
    #print(U)
    #print(V)
    for epoch in range(num_epochs):
        for r in range(m):
            for c in range(n):
                e = A[r][c] - torch.dot(U[r],V[c])
                U[r] += lr*e*V[c]
                V[c] += lr*e*U[r]
    return (U,V)
    
############
# PART 1.2 #
############

A = torch.tensor([[0.3374, 0.6005, 0.1735],
                  [3.3359, 0.0492, 1.8374],
                  [2.9407, 0.5301, 2.2620]])

U,V = sgd_factorise(A,2)
print("U:\n%s" % U)
print("V:\n%s" % V)
print("U*V^T:\n%s" % torch.mm(U,V.T))

print("MSE loss:")
print(torch.nn.functional.mse_loss(torch.mm(U,V.T), A, reduction="sum"))

U:
tensor([[ 0.6242, -0.2610],
        [ 0.8736,  1.5947],
        [ 1.4562,  1.0727]])
V:
tensor([[ 0.9855,  1.4963],
        [ 0.6882, -0.3813],
        [ 0.8856,  0.7569]])
U*V^T:
tensor([[ 0.2245,  0.5291,  0.3552],
        [ 3.2471, -0.0069,  1.9807],
        [ 3.0402,  0.5931,  2.1016]])
MSE loss:
tensor(0.1220)


In [2]:
############
# PART 2.1 #
############

U,S,V = torch.svd(A)
S2 = torch.tensor([[S[0],0,0],
                   [0,S[1],0],
                   [0,0,0]])


A2 = torch.mm(U, torch.mm(S2,V.T))
print("SVD reconstruction:")
print(A2)
print("MSE loss:",float(torch.nn.functional.mse_loss(A2, A, reduction="sum")))


S2[2][2] = 0
A2 = torch.mm(U,torch.mm(S2,V.T))
print("\nSVD reconstruction with last singular value = 0:")
print(A2)

print("MSE loss:",float(torch.nn.functional.mse_loss(A2, A, reduction="sum")))

SVD reconstruction:
tensor([[0.3374, 0.6005, 0.1735],
        [3.3359, 0.0492, 1.8374],
        [2.9407, 0.5301, 2.2620]])
MSE loss: 2.076908089954088e-12

SVD reconstruction with last singular value = 0:
tensor([[ 0.2245,  0.5212,  0.3592],
        [ 3.2530, -0.0090,  1.9737],
        [ 3.0378,  0.5983,  2.1023]])
MSE loss: 0.12191087752580643


In [10]:
############
# PART 3.2 #
############

import numpy as np

def sgd_factorise_masked(A:torch.Tensor, M:torch.Tensor, rank:int, num_epochs=1000, lr=0.01):
    m = A.shape[0]
    n = A.shape[1]
    U = torch.rand(m,rank)
    V = torch.rand(n,rank)
    for epoch in range(num_epochs):
        for r in range(m):
            for c in range(n):
                if M[r][c] == 1:
                    e = A[r][c] - torch.dot(U[r],V[c])
                    U[r] += lr*e*V[c]
                    V[c] += lr*e*U[r]
    return (U,V)
                    
                    
A = torch.tensor([[0.3374, 0.6005, 0.1735],
                  [0,      0.0492, 1.8374],
                  [2.9407, 0,      2.2620]])
M = torch.tensor([[1,1,1],
                  [0,1,1],
                  [1,0,1]])
U,V = sgd_factorise_masked(A,M,2)


############
# PART 3.2 #
############


print("U:\n%s" % U)
print("V:\n%s" % V)
print("U*V^T:\n%s" % torch.mm(U,V.T))

A_true = torch.tensor([[0.3374, 0.6005, 0.1735],
                       [3.3359, 0.0492, 1.8374],
                       [2.9407, 0.5301, 2.2620]])

print("MSE loss:")
print(torch.nn.functional.mse_loss(torch.mm(U,V.T), A_true, reduction="sum"))

U:
tensor([[-0.3289,  0.5210],
        [ 1.4669,  0.3542],
        [ 0.9201,  1.4150]])
V:
tensor([[ 1.0848,  1.3701],
        [-0.2097,  1.0086],
        [ 1.0232,  0.9374]])
U*V^T:
tensor([[0.3569, 0.5944, 0.1518],
        [2.0766, 0.0496, 1.8330],
        [2.9367, 1.2342, 2.2678]])
MSE loss:
tensor(2.0825)


In [8]:
print(torch.mm(U,V.T)-A_true)

tensor([[-3.0189e-03, -6.3227e-03,  3.5381e-03],
        [-8.7966e-01,  1.9038e-03, -7.7438e-04],
        [-3.4666e-04, -9.8404e-01,  6.4325e-04]])


In [ ]:
-0.0030 & -0.0063 & -0.0035\\
-0.8797 & -0.0019 & -0.0008\\
-0.0003 & -0.9840 & 0.0006